# 📊 Data Visualization - Olympics  

##### Data Science Degree da Let's Code - Módulo 5 - Técnicas de Programação II | Python
---

## 📰 Metadados

**Arquivo:**
```athlete_events.csv```.

**Formato:**
271.116 linhas e 15 colunas. Cada linha corresponde a um indivíduo (atleta) que competiu em um evento olímpico individual.

**Dataset:**

| Coluna | Descrição                                       |
|--------|-------------------------------------------------|
| ID     | Chave unica para cada atleta                    |
| Name   | Nome do atleta                                  |
| Sex    | Gênero do atleta (M ou F)                       |
| Age    | Idade do atleta no momento da competição        |
| Height | Altura do atleta (cm)                           |
| Weight | Peso do atleta (kg)                             |
| Team   | Nome do time                                    |
| NOC    | National Olympic Committee (código de 3 letras) |
| Games  | Ano e temporada dos jogos                       |
| Year   | Ano dos jogos                                   |
| Season | Temporada dos jogos (summer ou winter)          |
| City   | Cidade em que ocorreu os jogos                  |
| Sport  | Esporte praticado pelo atleta                   |
| Event  | Categoria/modalidade da competição              |
| Medal  | Medalha adquirida: Gold, Silver, Bronze ou NaN  |

## 🧰 Preparando a caixa de ferramentas

In [1]:
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.io as pio
import plotly.express as px

In [2]:
# Inicializando o Plotly em modo off-line
pyo.init_notebook_mode(connected=True)

# Definindo o template padrão no Plotly como Simple White
pio.templates.default = "simple_white"

In [3]:
# Lendo o Dataset
df = pd.read_csv('data/athlete_events.csv')

## 🇧🇷 O Brasil nas olímpiadas

**1.** Crie um DataFrame contendo apenas informações sobre atletas **brasileiros**. Exiba a quantidade de linhas desse novo DataFrame.

In [4]:
df_brazil = df[df['NOC'] == 'BRA'].copy()

print(f'O DataFrame contendo apenas informações sobre atletas brasileiros contém {df_brazil.shape[0]} linhas.')

O DataFrame contendo apenas informações sobre atletas brasileiros contém 3848 linhas.


**2. Será que existe alguma correlação entre a altura do atleta e seu desempenho, para algum esporte? Faça essa análise separadamente para cada gênero, e conclua.**

Gere um ou mais gráficos de sua escolha, em que seja possível comparar as alturas dos atletas de cada esporte, separando-os por gênero. Se necessário, crie colunas ou DataFrames auxiliares.

**Facilitador:** se quiser, analise apenas os esportes em que se tenha ao menos 5 atletas medalhistas e 5 atletas não medalhistas, para facilitar visualizações e conclusões.

Os dados de desempenho limitam-se a apresentar se o atleta conquistou ou não medalha (e qual medalha foi conquistada).

Nesse caso, usaremos o **facilitador** proposto,limitando os dados a esportes em que o Brasil conquistou pelo menos 5 medalhas e pelo menos 10 competidores.

In [5]:
# Agrupando os dados por esporte
df_brazil_grouped = df_brazil.groupby(by='Sport', as_index=False).count()

# Definindo os filtros
medals_filter = df_brazil_grouped['Medal'] >= 5
competitors_filter = df_brazil_grouped['ID'] >= 10

# Filtrando o DataFrame
sports_list = np.array(df_brazil_grouped[medals_filter & competitors_filter]['Sport'])

print(sports_list)

['Athletics' 'Basketball' 'Beach Volleyball' 'Boxing' 'Equestrianism'
 'Football' 'Judo' 'Sailing' 'Shooting' 'Swimming' 'Volleyball']


Esportes em que o Brasil conquistou pelo 5 medalhas:

- Athletics - Atletismo
- Basketball - Basquetebol
- Beach Volleyball - Vôleibol de praia
- Boxing - Boxe
- Equestrianism - Equitação
- Football - Futebol
- Judo - Judô
- Sailing - Vela
- Shooting - Tiro
- Swimming - Natação
- Volleyball - Vôleibol

Os esportes **Boxe** e **Judô** serão desconsiderados.

Boxe e Judô tem seus eventos/categorias divididas por peso - o que tem grande correlação com a altura. Nesse caso, seria necessário analisar cada categoria individualmente. Como nenhuma categoria possui mais de 5 medalhas conquistadas, os esportes serão desconsiderados.

O esporte **Atletismo** possui uma variabilidade muito alta entre suas modalidades, reunindo saltos, corridas de explosão e de resistência e lançamentos. Por isso, também será desconsiderado.

### 📝 Definindo os padrões para plotagem dos gráficos

In [6]:
# Dataframe
# Criando uma cópia do DataFrame para plotagem dos gráficos
df_brazil_for_plotting = df_brazil.copy()

# Criando uma nova coluna em português para a plotagem dos gráficos
values_info = {'Gold': 'Ouro',
               'Silver': 'Prata',
               'Bronze': 'Bronze',
               np.nan: 'Sem Medalha'}

df_brazil_for_plotting['Medalha'] = df_brazil_for_plotting['Medal'].map(values_info)

# Criando uma cópia do DataFrame para plotagem de gráficos agrupados por time
df_brazil_for_plotting_grouped = df_brazil_for_plotting.groupby(['Year','City','Sex', 
                                                                 'Medalha', 'Sport', 'Event'], 
                                                                as_index=False).mean()

# Definições de apresentação
color_info = {'Sem Medalha': 'seagreen',
              'Bronze': 'lightsalmon',
              'Prata': 'grey',
              'Ouro': 'gold'}

category_orders = {"Medalha": ["Sem Medalha", "Bronze", "Prata", "Ouro"],
                 "Sex": ["M", "F"]}

# Definições de Hover
# Para dados individuais (atletas)
athletes_chosen_columns = ('Height','Name','City', 'Year', 'Event')

athletes_chosen_columns_for_hover = dict(zip(df_brazil_for_plotting.columns,
                                    [True if column in athletes_chosen_columns else False 
                                     for column in df_brazil_for_plotting.columns]))

# Para dados coletivos (time)
teams_chosen_columns = ('Height', 'City', 'Year', 'Event')

teams_chosen_columns_for_hover = dict(zip(df_brazil_for_plotting_grouped.columns,
                                    [True if column in teams_chosen_columns else False 
                                     for column in df_brazil_for_plotting_grouped.columns]))

### 🏀 Basquetebol

In [7]:
# Definindo esporte a ser analisado
analyzed_sport = 'Basketball'

#### Análise por Atleta

In [8]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado
df_brazil_for_sport_plotting = df_brazil_for_plotting[df_brazil_for_plotting['Sport'] == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=athletes_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='🏀 Altura dos Atletas x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='%{customdata[1]}<br>'
                                'Altura: %{y}cm.<br>'
                                'Competindo em: %{customdata[10]}, %{customdata[8]}.<extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Análise por Time

In [9]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado por equipe
df_brazil_for_sport_plotting_grouped = df_brazil_for_plotting_grouped[df_brazil_for_plotting_grouped['Sport'] 
                                                                      == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting_grouped, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=teams_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='🏀 Altura Média do Time x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='Jogos: %{customdata[1]}, %{customdata[0]}.<br>'
                                'Altura média do time: %{y:.0f}cm.<br><extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura Média (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Conclusão

Na categoria masculina, notamos que jogadores mais baixos (para o padrão do esporte) tiveram um melhor desempenho - o que é bastante surpreendente para o Basquetebol. Inclusive, nenhum jogador brasileiro com mais de 2m de altura conquistou alguma medalha na modalidade.

Visualizando as médias dos times, notamos ainda melhor essa tendência: as medalhas estão concentradas nos times com as menores médias de altura. Isso se inverte na categoria feminina.

### 🏖 Voleibol de Praia

In [10]:
# Definindo esporte a ser analisado
analyzed_sport = 'Beach Volleyball'

#### Análise por Atleta

In [11]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado
df_brazil_for_sport_plotting = df_brazil_for_plotting[df_brazil_for_plotting['Sport'] == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=athletes_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='🏖 Altura dos Atletas x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='%{customdata[1]}<br>'
                                'Altura: %{y}cm.<br>'
                                'Competindo em: %{customdata[10]}, %{customdata[8]}.<extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Conclusão

Não visualizamos alguma tendência entre altura e desempenho em medalhas.

### 🏇🏻 Equitação

In [12]:
# Definindo esporte a ser analisado
analyzed_sport = 'Equestrianism'

#### Análise por Atleta

In [13]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado
df_brazil_for_sport_plotting = df_brazil_for_plotting[df_brazil_for_plotting['Sport'] == analyzed_sport]

# Plotando gráfico para observações do genêro masculino
fig = px.strip(data_frame=df_brazil_for_sport_plotting[df_brazil_for_sport_plotting['Sex'] == 'M'], 
               x='Medalha', y='Height', 
               color='Medalha',
               hover_data=athletes_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='🏇 Altura dos Atletas (Masculinos) x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='%{customdata[1]}<br>'
                                'Altura: %{y}cm.<br>'
                                'Competindo em: %{customdata[10]}, %{customdata[8]}.<extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Adicionando aviso sobre o gênero apresentado
fig.add_annotation(text='<i>Aviso: Não houveram atletas femininas vencedoras de medalhas.</i>',
                   font_color='grey',
                   xref='paper', yref='paper',
                   x=0.5, y=-0.2, showarrow=False)

# Apresentando gráfico
fig.show()

#### Conclusão

Não visualizamos alguma tendência entre altura e desempenho em medalhas.

### ⚽️ Futebol

In [14]:
# Definindo esporte a ser analisado
analyzed_sport = 'Football'

#### Análise por Atleta

In [15]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado
df_brazil_for_sport_plotting = df_brazil_for_plotting[df_brazil_for_plotting['Sport'] == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=athletes_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='⚽️ Altura dos Atletas x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='%{customdata[1]}<br>'
                                'Altura: %{y}cm.<br>'
                                'Competindo em: %{customdata[10]}, %{customdata[8]}.<extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Análise por Time

In [16]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado por equipe
df_brazil_for_sport_plotting_grouped = df_brazil_for_plotting_grouped[df_brazil_for_plotting_grouped['Sport'] 
                                                                      == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting_grouped, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=teams_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='⚽️ Altura Média do Time x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='Jogos: %{customdata[1]}, %{customdata[0]}.<br>'
                                'Altura média do time: %{y:.0f}cm.<br><extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura Média (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Conclusão

Visualizamos (principalmente na categoria masculina) uma leve tendência de melhor desempenho conforme maior a altura do jogador. Isso se reforça no gráfico de times, onde a concentração de medalhas fica na parte superior do eixo Y.

### ⛵️ Vela

In [17]:
# Definindo esporte a ser analisado
analyzed_sport = 'Football'

#### Análise por Atleta

In [18]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado
df_brazil_for_sport_plotting = df_brazil_for_plotting[df_brazil_for_plotting['Sport'] == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=athletes_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='⛵️ Altura dos Atletas x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='%{customdata[1]}<br>'
                                'Altura: %{y}cm.<br>'
                                'Competindo em: %{customdata[10]}, %{customdata[8]}.<extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Conclusão

Não visualizamos alguma tendência entre altura e desempenho em medalhas.

### 🔫 Tiro

In [19]:
# Definindo esporte a ser analisado
analyzed_sport = 'Shooting'

#### Análise por Atleta

In [20]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado
df_brazil_for_sport_plotting = df_brazil_for_plotting[df_brazil_for_plotting['Sport'] == analyzed_sport]

# Plotando gráfico para observações do genêro masculino
fig = px.strip(data_frame=df_brazil_for_sport_plotting[df_brazil_for_sport_plotting['Sex'] == 'M'], 
               x='Medalha', y='Height', 
               color='Medalha',
               hover_data=athletes_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='🔫 Altura dos Atletas (Masculinos) x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='%{customdata[1]}<br>'
                                'Altura: %{y}cm.<br>'
                                'Competindo em: %{customdata[10]}, %{customdata[8]}.<extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Adicionando aviso sobre o gênero apresentado
fig.add_annotation(text='<i>Aviso: Não houveram atletas femininas vencedoras de medalhas.</i>',
                   font_color='grey',
                   xref='paper', yref='paper',
                   x=0.5, y=-0.2, showarrow=False)

# Apresentando gráfico
fig.show()

#### Conclusão

Não visualizamos alguma tendência entre altura e desempenho em medalhas.

### 🏊‍♂️ Natação

In [21]:
# Definindo esporte a ser analisado
analyzed_sport = 'Swimming'

#### Análise por Atleta

In [22]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado
df_brazil_for_sport_plotting = df_brazil_for_plotting[df_brazil_for_plotting['Sport'] == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=athletes_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='🏊‍♂️ Altura dos Atletas x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='%{customdata[1]}<br>'
                                'Altura: %{y}cm.<br>'
                                'Competindo em: %{customdata[10]}, %{customdata[8]}.<extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Conclusão

Para a categoria masculina, também visualizamos uma concentração de medalhas na parte superior, apesar de observamos um *outlier* de 1.58m, todos os atletas que conquistaram medalha tinham uma altura superior a 1.80m.

Não observamos correlação na categoria feminina.

### 🏐 Voleibol

In [23]:
# Definindo esporte a ser analisado
analyzed_sport = 'Volleyball'

#### Análise por Atleta

In [24]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado
df_brazil_for_sport_plotting = df_brazil_for_plotting[df_brazil_for_plotting['Sport'] == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=athletes_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='🏐 Altura dos Atletas x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='%{customdata[1]}<br>'
                                'Altura: %{y}cm.<br>'
                                'Competindo em: %{customdata[10]}, %{customdata[8]}.<extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Análise por Time

In [25]:
# Filtrando o DataFrame para plotar o gráfico do esporte analisado por equipe
df_brazil_for_sport_plotting_grouped = df_brazil_for_plotting_grouped[df_brazil_for_plotting_grouped['Sport'] 
                                                                      == analyzed_sport]

# Plotando gráfico
fig = px.strip(data_frame=df_brazil_for_sport_plotting_grouped, x='Medalha', y='Height', 
               color='Medalha', 
               facet_col='Sex',
               hover_data=teams_chosen_columns_for_hover,
               category_orders=category_orders,
               color_discrete_map=color_info,
               title='🏐 Altura Média do Time x Performance em Medalhas')

# Customizando gráfico
fig.update_traces(hovertemplate='Jogos: %{customdata[1]}, %{customdata[0]}.<br>'
                                'Altura média do time: %{y:.0f}cm.<br><extra></extra>')

fig.update_layout(yaxis=dict(tickfont_size=10, 
                             title=dict(font_size=12, 
                                        text='Altura Média (cm)')))

fig.update_xaxes(matches=None,
                 tickfont_size=10, 
                 title=dict(font_size=12, 
                            text='Performance (Medalha)'))

fig.for_each_annotation(lambda annotation: annotation.update(text='👨 Masculino', font_size=13) 
                        if annotation.text.split("=")[-1] == 'M' 
                        else annotation.update(text='🙎‍♀️ Feminino', font_size=13))

# Apresentando gráfico
fig.show()

#### Conclusão

Aqui, principalmente na visualização de altura média do time, notamos que há uma boa correlação entre altura e desempenho para ambas as categorias.

Na visualização de atletas, os dados estão mais distribuídos. Porém, é importante levar em consideração que há atletas de posições naturalmente mais baixas (levantador e líbero). Mesmo assim, notamos uma correlação.

### 📝 Considerações Finais

Cada esporte tem suas peculiaridades, que são dinâmicas e evoluem no tempo. Encontramos comportamentos variados em cada visualização.

Para conclusões mais robustas, os dados de ranking final do atleta (incluindo quarto, quinto,..., último lugar) proveriam uma base sólida para melhores conclusões.

## 🏅Medalhas por Esporte

**3. Vamos analisar agora as medalhas que nossos atletas trouxeram para casa.**

Crie uma visualização de sua preferência para mostrar, por esporte, a proporção de medalhas de ouro, prata e bronze que nosso país já consquistou.

**Facilitador:** Se preferir, mostre as proporções apenas para os 6 esportes que tiveram mais medalhas.  

*DICA:* Cuidado! Esportes em equipe dão medalhas a todos os jogadores, mas só contabilizam uma vez!

In [26]:
# Criando uma cópia do DataFrame para plotagem do gráfico de medalhas
df_brazil_medal_plot = df_brazil.copy()

# Agrupando o DataFrame para obter 01 registro por medalha conquistada
df_brazil_medal_plot = df_brazil_medal_plot.groupby(by=['Medal', 'Sport',
                                                        'Event', 'Year'], 
                                                    as_index=False).count()

# Agrupando o DataFrame por Medalha e Esporte
df_brazil_medal_plot = df_brazil_medal_plot.groupby(by=['Medal', 'Sport'], as_index=False).count()

# Criando uma nova coluna em português para a plotagem das medalhas no gráfico
values_info = {'Gold': 'Ouro',
               'Silver': 'Prata',
               'Bronze': 'Bronze'}

df_brazil_medal_plot['Medalha'] = df_brazil_medal_plot['Medal'].map(values_info)

# Criando uma nova coluna em português para a plotagem dos esportes nos gráficos
sports_info = {'Sailing': 'Vela',
               'Athletics': 'Atletismo',
               'Volleyball': 'Voleibol',
               'Judo': 'Judô',
               'Beach Volleyball': 'Voleibol de Praia',
               'Boxing': 'Boxe',
               'Equestrianism' : 'Equitação',
               'Football': 'Futebo',
               'Gymnastics': 'Ginástica',
               'Shooting': ' Tiro',
               'Swimming': 'Natação',
               'Canoeing': 'Canoagem',
               'Basketball': 'Basquetebol',
               'Taekwondo': 'Taekwondo',
               'Modern Pentathlon': 'Pentlato Moderno'}

df_brazil_medal_plot['Esporte'] = df_brazil_medal_plot['Sport'].map(sports_info)

# Definindo a sequência de ordenação das colunas
df_brazil_medal_plot['Medalha'] = pd.Categorical(values=df_brazil_medal_plot['Medalha'], 
                                categories=('Bronze', 'Prata', 'Ouro'), 
                                ordered=True)

df_brazil_medal_plot.sort_values(['Medalha', 'Event'], ascending=False, inplace=True)

# Definições de apresentação
color_info = {'Bronze': 'lightsalmon',
              'Prata': 'grey',
              'Ouro': 'gold'}

# Plotando gráfico
fig = px.bar(data_frame=df_brazil_medal_plot, x='Event', y='Esporte', color='Medalha',
             color_discrete_map=color_info,
             orientation='h', text_auto=True,
             title='🏅 Medalhas Olímpicas por Esporte')

# Customizando gráfico
fig.update_traces(textposition='inside',
                  insidetextanchor='middle',
                  textangle=0)

fig.update_layout(yaxis=dict(autorange='reversed',
                             title_text=None,
                             ticks=''),
                  xaxis_visible=False,
                  hovermode=False)

# Apresentando gráfico
fig.show()

## 📈 Série Temporal de Medalhas 

**4. Para finalizar a história do Brasil, vamos ver a série temporal de medalhas brasileiras.**

Crie um gráfico de sua escolha, que mostre o total de medalhas de ouro, prata e bronze, por edição da olimpíada (em ordem cronológica). 

*DICA:* Mais uma vez, tome cuidado com os esportes em grupo - os contabilize apenas uma vez!

In [36]:
# Determinando o país a ser analisado pelo NOC
chosen_country = 'BRA'

# Determinando se as Olímpiadas de Inverno serão consideradas
include_winter = False

# Aplicando os filtros
df_time_plot = df[df['NOC'] == chosen_country].copy()

if not include_winter:
    df_time_plot = df_time_plot[df_time_plot['Season'] != 'Winter'].copy()

# Criando uma nova coluna em português para a contagem das medalhas
values_info = {'Gold': 'Ouro',
               'Silver': 'Prata',
               'Bronze': 'Bronze',
               np.nan: 'Sem Medalha'}

df_time_plot['Medalha'] = df_time_plot['Medal'].map(values_info)

# Agrupando o DataFrame para contar somente 01 medalha por time
df_time_plot = df_time_plot.groupby(by=['Year', 'Medalha', 'Event'], as_index=False).count()    
df_time_plot = df_time_plot.groupby(by=['Year', 'Medalha'], as_index=False).count()    

# Obtendo os Dummies das Medalhas
df_time_plot_dummies = pd.get_dummies(df_time_plot['Medalha'])

# Reconectando o ano com a contagem de medalhas
df_time_plot = pd.concat([df_time_plot, df_time_plot_dummies.set_index(df_time_plot.index)], axis=1) 


df_time_plot



,Year,Medalha,Event,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Season,City,Sport,Medal,Bronze,Ouro,Prata,Sem Medalha
0,1900,Sem Medalha,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,1
1,1920,Bronze,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0
2,1920,Ouro,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0
3,1920,Prata,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0
4,1920,Sem Medalha,9,9,9,9,9,9,9,9,9,9,9,9,9,9,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2012,Sem Medalha,125,125,125,125,125,125,125,125,125,125,125,125,125,125,0,0,0,1
59,2016,Bronze,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,0,0,0
60,2016,Ouro,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,1,0,0
61,2016,Prata,6,6,6,6,6,6,6,6,6,6,6,6,6,6,0,0,1,0


In [ ]:
# Criando uma cópia do DataFrame para plotagem do gráfico de medalhas
df_brazil_time_plot = df_brazil.copy()

# Agrupando o DataFrame para obter 01 registro por medalha conquistada
df_brazil_time_plot = df_brazil_time_plot.groupby(by=['Year', 'Medal',
                                                       'Sport', 'Event'], 
                                                    as_index=False).count()

# Agrupando o DataFrame por Medalha e Esporte
df_brazil_time_plot = df_brazil_time_plot.groupby(by=['Year', 'Medal'], 
                                                  as_index=False).count()

df_brazil_time_plot.get_dummies('Medal')
# fig = px.line(data_frame=df_brazil_time_plot, x='Year', y='Sport', color='Medal', markers=True)

# fig.update_layout(xaxis_dtick='4')

# fig.show()